In [269]:
import sqlite3
import re

import plotly.io as pio
import plotly.express as px
import pandas as pd

pio.templates.default = "plotly_dark"

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("plotting.backend", "plotly")

In [270]:
grades = pd.read_json("grades.json")
grades.index = pd.MultiIndex.from_tuples([eval(t) for t in grades.index], names=["Level", "Subject"])
grades = grades.reset_index(0)

# Add columns
grades["الدرجات"] = grades["النقاط"] / grades["الساعات"] / 5 * 100
grades["قسم المقرر"] = grades["رمز المقرر"].apply(lambda x: re.sub(r"[0-9]* ", "", x))
symbol2category = {
    "ريد": "إنسانيات",
    "احص": "رياضيات",
    "كيم": "علوم",
    "انجل": "إنسانيات",
    "عرب": "إنسانيات",
    "تقن": "علوم",
    "نهج": "إنسانيات",
    "ريض": "رياضيات",
    "فجب": "إنسانيات",
    "فيز": "علوم",
    "عال": "علوم حاسب",
    "سلم": "سلم",
}
grades["صنف المقرر"] = grades["قسم المقرر"].apply(symbol2category.get)

# Addtional variables
ordering = {"التقدير": ["أ+", "أ", "ب+", "ند"]}

In [271]:
db = sqlite3.connect("db.sqlite3")

commits = pd.read_sql_query("""
                    SELECT
                    *
                    FROM commits
                """, db, parse_dates=["date"])
commits.columns = ["Repository", "Date", "Raw"]
commits = commits.sort_values("Date").reset_index(drop=True)
commits["Commit"] = 1

In [272]:
px.strip(commits, x="Date", y="Repository", color="Repository", height=1000, width=1300) 

In [273]:
px.bar(grades, y="التقدير", color="صنف المقرر", text="اسم المقرر", category_orders=ordering)

In [274]:
px.bar(grades, x="الترم", y="الدرجات", color="صنف المقرر", text="اسم المقرر", category_orders=ordering)

In [275]:
px.bar(grades, x="صنف المقرر", y="الدرجات", color="التقدير", text="اسم المقرر", barmode="group", category_orders=ordering)